In [ ]:
#Importing necessary libraries for the project
import numpy as np  # Importing NumPy for numerical operations and array manipulations
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting graphs and visualizations
import seaborn as sns  # Importing Seaborn for statistical data visualization, built on top of Matplotlib
import tensorflow as tf  # Importing TensorFlow for building and training machine learning models
from tensorflow import keras  # Importing Keras, a high-level API for TensorFlow, to simplify model building
from tensorflow.keras import Layer  # Importing Layer class for creating custom layers in Keras
from tensorflow.keras.models import Sequential  # Importing Sequential model for building neural networks layer-by-layer
from tensorflow.keras.layers import Rescaling , GlobalAveragePooling2D
from tensorflow.keras import layers, optimizers, callbacks  # Importing various modules for layers, optimizers, and callbacks in Keras
from sklearn.utils.class_weight import compute_class_weight  # Importing function to compute class weights for imbalanced datasets
from tensorflow.keras.applications import EfficientNetV2B2  # Importing EfficientNetV2S model for transfer learning
from sklearn.metrics import confusion_matrix, classification_report  # Importing functions to evaluate model performance
import gradio as gr


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your dataset zip (e.g., trash_dataset.zip)


In [ ]:
!unzip archive\ \(3\).zip -d dataset/


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/TrashType_Image_Dataset",   # ✅ updated path
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(180, 180),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/TrashType_Image_Dataset",   # ✅ updated path
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(180, 180),
    batch_size=32
)


In [ ]:
val_class=val_ds.class_names
print(val_class)

In [ ]:
val_batches=tf.data.experimental.cardinality(val_ds)
print(val_batches)



In [ ]:
test_ds=val_ds.take(val_batches //2)
val_dat=val_ds.skip(val_batches //2)
test_ds_eval=test_ds.cache().prefetch(tf.data.AUTOTUNE)
print(train_ds.class_names)
print(val_class)
print(len(train_ds.class_names))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_ds.class_names[labels[i]])
        plt.axis("off")

In [ ]:
def count_distribution(dataset, class_names):
    total = 0
    counts = {name: 0 for name in class_names}

    for _, labels in dataset:
        for label in labels.numpy():
            class_name = class_names[label]
            counts[class_name] += 1
            total += 1

    for k in counts:
        counts[k] = round((counts[k] / total) * 100, 2)  # Convert to percentage
    return counts

In [ ]:
def simple_bar_plot(dist, title):
    plt.bar(dist.keys(), dist.values(), color='cornflowerblue')
    plt.title(title)
    plt.ylabel('Percentage (%)')
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()

In [ ]:
# Ensure datasets are defined
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/TrashType_Image_Dataset",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(180, 180),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/TrashType_Image_Dataset",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(180, 180),
    batch_size=32
)

# Recalculate val_batches and test_ds after re-creating val_ds
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds_split = val_ds.take(val_batches // 2)
val_dat = val_ds.skip(val_batches // 2)

class_names=train_ds.class_names
train_dist=count_distribution(train_ds,class_names)
val_dist=count_distribution(val_ds,class_names)
test_dist = count_distribution(test_ds_split, class_names)

overall_dist={}
for k in class_names:
  overall_dist[k] = round((train_dist[k] + val_dist[k]) / 2, 2)

print("Training Distribution:", train_dist)
print("Validation Distribution:", val_dist)
print("Test Distribution:", test_dist)
print("Overall Distribution:", overall_dist)

In [ ]:
class_counts={i:0 for i in range(len(class_names))}
all_labels=[]
for images,labels in train_ds:
  for label in labels.numpy():
    class_counts[label]+=1
    all_labels.append(label)

class_weights_array=compute_class_weight(class_weight='balanced',classes=np.arange(len(class_names)),y=all_labels)
class_weights={i: w for i,w in enumerate(class_weights_array)}
print(class_counts)
print(class_weights)

In [ ]:
data_augmentation=Sequential([layers.RandomFlip("horizontal"),layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),])


In [ ]:
base_model=EfficientNetV2B2(include_top=False,weights='imagenet',input_shape=(180,180,3),include_preprocessing=True)
base_model.trainable = True
for layer in base_model.layers[:100]:  # You can adjust this number
    layer.trainable = False


In [ ]:
model=Sequential([layers.Input((180,180,3)),data_augmentation,base_model,GlobalAveragePooling2D(),layers.Dropout(0.3),layers.Dense(len(class_names),activation='softmax')])
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
epochs=3
history=model.fit(train_ds,validation_data=val_ds,epochs=epochs,class_weight=class_weights,batch_size=32,callbacks=[early])
model.summary()
base_model.summary()

In [ ]:
model.save("week2_model.keras")